In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Initialization

In [3]:
import numpy as np
import random
from scipy.spatial import distance as d
import os
import math
import pandas as pd
import time
import json

TEST_DIR = '/content/gdrive/My Drive/test-folder/'

LEAF_FOLDER = '/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests'

TEST_FILE_1 = os.path.join(TEST_DIR, "test-tree-construction-data.npy")
TEST_FILE_2 = os.path.join(TEST_DIR, "test-tree-construction-data-2.npy")
TEST_FILE_3 = os.path.join(TEST_DIR, "test-tree-construction-data-3.npy")


TEST_PATH = "/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests"

FEATURES_PATH = "/content/gdrive/My Drive/[MIRCV]FoodWebSearch/features.csv"
FEATURES_PATH_TEST_1 = "/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/features-test-1.npy"
FEATURES_NAMES_TEST_1 = "/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/features-names-test-1.npy"

FEATURES_PATH_TEST_2 = "/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/features-test-2.npy"
FEATURES_NAMES_TEST_2 = "/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/features-names-test-2.npy"

FEATURES_PATH_TEST_3 = "/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/features-test-3.npy"
FEATURES_NAMES_TEST_3 = "/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/features-names-test-3.npy"


ID_DEPLOYMENT = "/content/gdrive/MyDrive/[MIRCV]FoodWebSearch/id.npy"
FEATURES_DEPLOYMENT = "/content/gdrive/MyDrive/[MIRCV]FoodWebSearch/features.npy"
INDEX_DIR = "/content/gdrive/MyDrive/[MIRCV]FoodWebSearch/deployment"

Create the datasets for tests

In [4]:
np.save(FEATURES_PATH_TEST_2, [[63, 27, 38], [60.21, 24, 26], [60, 82, 0], [19, 39, 52], [10, 57, 98], 
                      [46, 19, 41], [32, 13, 6], [14, 74, 12], [98, 27, 22], [59, 62, 0], 
                      [4, 9, 22], [62, 56, 63], [38, 86, 60], [66, 13, 90], [95, 56, 70], 
                      [11, 71, 14], [56, 97, 40], [94, 6, 52], [63, 67, 91], [88, 70, 15], 
                      [72, 98, 32], [66, 61, 89], [92, 86, 13], [40, 97, 35], [26, 3, 10], 
                      [40, 61, 99], [49, 83, 24], [54, 32, 22], [70, 42, 55], [43, 34, 61]])
np.save(FEATURES_NAMES_TEST_2, ["img_" + str(i) for i in range(30)])

In [5]:
np.save(FEATURES_NAMES_TEST_3, [[56, 0, 26, 95], [73, 40, 13, 47], [46, 81, 16, 78], [82, 9, 83, 32], [65, 59, 85, 30], 
                      [88, 49, 11, 19], [65, 76, 27, 58], [17, 84, 91, 33], [93, 80, 67, 64], [58, 91, 75, 38], 
                      [46, 69, 2, 49], [26, 39, 35, 70], [35, 95, 88, 90], [51, 84, 68, 72], [93, 75, 24, 88], 
                      [77, 82, 87, 12], [13, 16, 93, 12], [35, 18, 61, 42], [58, 18, 29, 23], [51, 24, 26, 99], 
                      [20, 28, 34, 12], [40, 21, 62, 14], [78, 79, 18, 23], [99, 22, 48, 79], [20, 73, 6, 64], 
                      [41, 14, 35, 67], [20, 4, 73, 74], [50, 84, 90, 12], [86, 64, 36, 58], [11, 51, 70, 32], 
                      [62, 91, 39, 84], [82, 28, 35, 29], [38, 22, 40, 67], [45, 51, 30, 44], [98, 2, 0, 49], 
                      [72, 57, 87, 80], [42, 32, 19, 93], [51, 50, 73, 6], [63, 40, 50, 75], [10, 84, 56, 27], 
                      [14, 34, 30, 85], [55, 44, 96, 93], [37, 73, 83, 42], [51, 4, 25, 8], [42, 10, 19, 82], 
                      [11, 29, 18, 31], [19, 28, 47, 39], [4, 69, 25, 28], [78, 12, 30, 48], [87, 28, 10, 24], 
                      [25, 93, 39, 97], [4, 62, 39, 89], [61, 29, 38, 82], [53, 49, 72, 46], [22, 86, 7, 50], 
                      [14, 92, 74, 65], [91, 67, 45, 21], [69, 12, 60, 52], [80, 3, 89, 93], [26, 70, 8, 72], 
                      [44, 36, 38, 92], [99, 25, 15, 49], [48, 79, 78, 38], [35, 67, 33, 53], [80, 68, 43, 41], 
                      [28, 19, 69, 37], [53, 58, 0, 17], [69, 7, 62, 73], [43, 34, 30, 29], [43, 41, 27, 57], 
                      [17, 59, 84, 91], [65, 55, 7, 82], [48, 3, 49, 99], [63, 44, 62, 9], [48, 53, 26, 39], 
                      [2, 17, 25, 31], [64, 30, 95, 15], [82, 76, 88, 15], [53, 31, 68, 76], [7, 36, 70, 71], 
                      [10, 31, 30, 25], [8, 43, 98, 25], [46, 97, 90, 19], [38, 2, 36, 51], [56, 24, 31, 97], 
                      [77, 45, 2, 69], [80, 52, 44, 10], [75, 84, 22, 51], [40, 49, 24, 65], [46, 32, 44, 66], 
                      [33, 13, 16, 34], [50, 20, 29, 52], [85, 54, 43, 78], [12, 19, 14, 64], [22, 74, 13, 94], 
                      [14, 26, 10, 15], [81, 25, 41, 86], [74, 23, 79, 54], [49, 2, 60, 7], [68, 26, 88, 75]])
np.save(FEATURES_NAMES_TEST_3, ["img_" + str(i) for i in range(100)])

# Create Tree Data Structure

##The Node
The class Node represents a node in the tree. 
We can have two types of nodes:
- internal node
- leaf node

All the nodes have an id, in order to store the tree on disk, preserving its structure.

####Internal Node
In the case of an internal node, the class has the following parameters:
- reference to the right child
- reference to the left child
- the value of the node, it's the pivot in this application: a numpy array that represent a point in the space
- the median: the value of the median is computed on the sorted distances between the pivot and the other values of the set

####Leaf Node

In the case of a leaf node, the class has the following fields:
- the pivot
- the median
- reference to the file in which are stored the objects on the left
- reference to the file in which are stored the objects on the right

In [6]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, None):
            return ""
        return json.JSONEncoder.default(self, obj)

In [7]:
class Node:

  def __init__(self, id, is_leaf, **kwargs):
    self.parent = kwargs.get("parent", None)
    self.id = id
    self.is_leaf = is_leaf
    self.pivot = kwargs.get("pivot", None)
    self.median = kwargs.get("median", -1)
    if self.is_leaf:
      self.objects = kwargs.get("objects", [])
      self.file_path_s_1, self.file_path_s_2 = "", ""
    else:
      self.right = kwargs.get("right", None)
      self.left = kwargs.get("left", None)

  def set_parameters(self, pivot, median):
    self.pivot = pivot
    self.median = median

  def add_children(self, left, right):
    self.left = left
    self.right = right

  def add_objects(self, s_1, s_2):
    self.objects_left = s_1
    self.objects_right = s_2

  def save_leaf_objects_on_disk(self, file_path, s_1, s_2):
    self.file_path_s_1 = file_path + "_subset_1.npy"
    self.file_path_s_2 = file_path + "_subset_2.npy"
    np.save(self.file_path_s_1, np.array(s_1, dtype=object))
    np.save(self.file_path_s_2, np.array(s_2, dtype=object))

  def load_objects_from_disk(self, left=True, right=True):
    if left and not right:
      result = np.load(self.file_path_s_1, allow_pickle=True)
      return result
    if right and not left:
      result = np.load(self.file_path_s_1, allow_pickle=True)
      return result
    s_1 = np.load(self.file_path_s_1, allow_pickle=True)
    s_2 = np.load(self.file_path_s_2, allow_pickle=True)
    result = np.concatenate((s_1, s_2))
    return result

  def get_node_name(self):
    return self.id

##The Vantage Point Tree
The Vantage Point Tree is modeled as a class, that provides us the methods to:
- create the tree from a dataset
- search the k-nearest neighbors objects, given a query
- print the tree<br>

All these methods are based on recursion.<br>
The Vantage Point Tree can be created in two modes:
1. Test mode: in order to test the tree on very small datasets
2. Disk mode: in this mode we can handle very huge datasets, by storing the objects on disk and keeping in the tree just the reference to the files

### Save Tree on disk

The save method use the recursion. For each node, a json element is created, each element contains:
- node id
- boolean to recognize a leaf
- the pivot as numpy array

If we deal with an internal node, we have:
- left child id
- right child id

On the other hand, the leaf node has:
- path of the file containing the objects on the left
- path of the file containing the objects on the right

At the end we will have a json array, each entry represents a node.
This json is saved on disk.

### Load Tree from disk

The load method works in the opposite way with respect to save. 
It also uses recursion:
1. the algorithm loads and deserializes the json
2. it finds the node in the json array using the id (node has id "0")
3. it builds a node object from the json
4. it finds the left child through the left child id in the entry
5. it finds the right child through the right child id in the entry
6. it builds the nodes recursively until it reaches the leaves

In [28]:
class VP_Tree:

  def __init__(self, index_name, height, disk_mode=True, leaves_path=None):
    self.root = None
    self.index_name = index_name
    self.height = height #to review
    self.disk_mode = disk_mode
    self.leaves_path = leaves_path

  def create_vptree(self, names_path, features_path):
    start = time.time()
    data = VP_Tree.read_data(names_path, features_path)
    n = len(data)
    print("Number of data:", n)
    max_height = math.floor(math.log(n,2)-1)
    print("The max height of the tree is:", max_height)
    if self.height > max_height: self.height = max_height 
    #take 1 pivot randomly and set pivot as root
    self.root, s_1, s_2 = self.partition_by_median(data)
    self.create_tree_level(self.root, s_1, s_2, 1)
    end = time.time()
    print("Building of the tree completed in:", end-start, "s")
  
  def create_tree_level(self, node, s_1, s_2, iteration):
      print("Level n." + str(iteration))
      is_leaf = iteration + 1 >= self.height
      left_node, s_1_left, s_2_left = self.partition_by_median(s_1, parent=node,is_left=True, is_leaf=is_leaf)
      right_node, s_1_right, s_2_right = self.partition_by_median(s_2, parent=node,is_left=False, is_leaf=is_leaf)
      node.add_children(right_node, left_node)
      if iteration + 1 < self.height:
        self.create_tree_level(left_node, s_1_left, s_2_left, iteration + 1)
        self.create_tree_level(right_node, s_1_right, s_2_right, iteration + 1)
      else:
        if self.disk_mode:
          left_path = self.get_leaves_path(left_node.get_node_name())
          right_path = self.get_leaves_path(right_node.get_node_name())
          left_node.save_leaf_objects_on_disk(left_path, s_1_left, s_2_left)
          right_node.save_leaf_objects_on_disk(right_path, s_1_right, s_2_right)
        else:
          left_node.add_objects(s_1_left, s_2_left)
          right_node.add_objects(s_1_right, s_2_right)

  def partition_by_median(self, data, parent=None,is_left=False,is_leaf=False):
    pivot_index = random.choice(range(len(data)))
    pivot = data[pivot_index]
    del data[pivot_index]
    #compute all the distances
    data = [value for value in data if not np.array_equal(value[1], pivot[1])] # refactor
    distances = np.array([d.euclidean(pivot[1],element[1]) for element in data])
    #sort the distances
    zipped_data_distances = sorted(zip(data, distances), key= lambda x:x[1])
    ordered_data, distances = zip(*zipped_data_distances)
    median = np.median(distances)
    #get the median
    s_1 = [element for element, distance in zipped_data_distances if distance <= median]
    s_2 = [element for element, distance in zipped_data_distances if distance >= median]
    #update node
    if parent == None:
      node = Node(id="0", is_leaf=is_leaf, pivot=pivot, median=median)
    else:
      node_id = parent.id + str(0 if is_left else 1)
      node = Node(node_id, is_leaf=is_leaf, pivot=pivot, median=median)
    return node, s_1, s_2

  def save_vptree(file_path, tree):
    if not os.path.exists(file_path): os.mkdir(file_path)
    file = os.path.join(file_path, tree.index_name + '.json')
    if os.path.exists(file):
      os.remove(file)
    with open(file, 'a') as json_file:
      index_json = {"index": tree.index_name, "nodes":[], "height":tree.height}
      VP_Tree.save_node(tree.root, index_json)
      vp_tree_json = json.dumps(index_json, cls=NumpyEncoder)
      json_file.write(vp_tree_json)
      print("File saved correctly in:", file)
    return file
  
  def save_node(node, index_json):
    if node.is_leaf:
        row_json={"is_leaf":True, 
                    "id":node.id,
                    "pivot" : node.pivot,
                    "median":node.median, 
                    "left_file":node.file_path_s_1, 
                    "right_file":node.file_path_s_2}
        index_json["nodes"].append(row_json)
    else:
        row_json={"is_leaf":False,
                  "id":node.id, 
                  "pivot": node.pivot,
                  "median": node.median,
                  "right_child":node.right.id,
                  "left_child":node.left.id}
        index_json["nodes"].append(row_json)
        VP_Tree.save_node(node.left, index_json)
        VP_Tree.save_node(node.right,index_json)
    return

  def load_vptree(path):
    if not os.path.exists:
      print("the path do not exist")
      return None
    entry_list=[]
    with open(path,'r', encoding='utf-8') as f:
      json_tree = json.load(f)
      entry_list=json_tree["nodes"]
    root_node=VP_Tree.parse_node('0',entry_list)
    index_name = json_tree["index"]
    height = json_tree["height"]
    vp_tree = VP_Tree(index_name=index_name,height=height,leaves_path=path)
    vp_tree.root = root_node
    print("Tree loaded correctly")
    return vp_tree

  def parse_node(id, nodes):
    node_json = None
    for element in nodes:
      if element["id"]==id:
        node_json = element
    node=Node(id=node_json["id"], is_leaf=node_json["is_leaf"], 
              pivot=node_json["pivot"], median=node_json["median"])
    if (node.is_leaf):
      node.file_path_s_1=node_json["left_file"]
      node.file_path_s_2=node_json["right_file"]
    else:
      right=VP_Tree.parse_node(node_json["right_child"],nodes)
      left=VP_Tree.parse_node(node_json["left_child"],nodes)
      node.add_children(left, right)
    return node

  def knn_search(self, k, query):
    start = time.time()
    nn = [None for i in range(k)]
    d_nn = [math.inf for i in range(k)]
    nn, d_nn = self.search_subtree(self.root, nn, d_nn, k, query)
    end = time.time()
    print("Query", query, "answered in", end-start, " s")
    return self.reorder_list_on_distances(nn, d_nn, desc=False)

  def search_subtree(self, node, nn, d_nn, k, query):
    pivot, median = node.pivot, node.median
    distance = d.euclidean(pivot[1], query)
    if distance <= d_nn[0]:
      d_nn[0] = distance
      nn[0] = pivot
      nn, d_nn = self.reorder_list_on_distances(nn, d_nn)
    if node.is_leaf:
      return self.search_in_leaf(node, nn, d_nn, k, query)
    if distance - d_nn[0] <= median:
      nn, d_nn = self.search_subtree(node.left, nn, d_nn, k, query)
    if distance + d_nn[0] >= median:
      nn, d_nn = self.search_subtree(node.right, nn, d_nn, k, query)
    return nn, d_nn

  def search_in_leaf(self, node, nn, d_nn, k, query):
    objects = []
    distance_pivot = d.euclidean(node.pivot[1], query)
    left, right = False, False
    if self.disk_mode:
      if distance_pivot - d_nn[0] <= node.median: left = True
      if distance_pivot + d_nn[0] >= node.median: right = True
      objects = node.load_objects_from_disk(left=left, right=right)
    else:
      objects = node.objects_left + node.objects_right
    for obj in objects:
      distance = d.euclidean(obj[1], query)
      if distance < d_nn[0]:
        nn[0] = obj
        d_nn[0] = distance
        nn, d_nn = self.reorder_list_on_distances(nn, d_nn)
    return nn, d_nn

  def reorder_list_on_distances(self, nn, d_nn, desc=True):
      zipped = sorted(zip(nn, d_nn), key= lambda x:x[1], reverse=desc)
      nn, d_nn = zip(*zipped)
      return list(nn), list(d_nn)

  def print_tree(node, level, disk_mode=True):
    indentation = "\n" + str(level * "\t")
    response = "id: " + node.id + " " + str(node.pivot)
    if node.is_leaf:
      if disk_mode: 
        response += indentation + str(node.file_path_s_1)
        response += indentation + str(node.file_path_s_2)
      else:
        response += indentation + str(node.objects_left)
        response += indentation + str(node.objects_right)
      return response
    response += indentation + VP_Tree.print_tree(node=node.right, level=level+1, disk_mode=disk_mode)
    response += indentation + VP_Tree.print_tree(node=node.left, level=level+1, disk_mode=disk_mode)
    return response

  def get_leaves_path(self, file_name):
    if not self.leaves_path is None:
      directory = os.path.join(self.leaves_path, self.index_name)
    else: directory = os.path.join(LEAF_FOLDER, self.index_name)
    if not os.path.exists(directory):
      os.mkdir(directory)
    leaves_directory = os.path.join(directory, "leaves_"+ str(self.height))
    if not os.path.exists(leaves_directory):
      os.mkdir(leaves_directory)
    return os.path.join(leaves_directory, file_name)

  def read_data(file_path_names, file_path_features):
    names = np.load(file_path_names)
    features = np.load(file_path_features)
    return [(name, feature) for name, feature in zip(names, features)]

## Tests
Tests on the creation use data with the following composition:
- test1: 20 3-dimensional points
- test2: 30 3-dimensional points
- test3: 100 4-dimensional points

### Tests on Partition by Median

In [ ]:
# Partition By Median Tests 1
if __name__ == '__main__':
  data = [("image0", np.array([0,2,1])) , ("image1",np.array([2,3,6])), ("image2",np.array([5,3,2])),
                  ("image3",np.array([5,6,4])), ("image4",np.array([5,16,1])), ("image5",np.array([2,6,2])), ("image6",np.array([1,3,1]))]

  vantage_point_tree = VP_Tree("Index_Test", 5)
  node, s_1, s_2 = vantage_point_tree.partition_by_median(data)

  print("Node:", node.pivot)
  print("Median:", node.median)
  print("Set 1:", s_1)
  print("Set 2:", s_2)

Node: ('image5', array([2, 6, 2]))
Median: 4.412608191037562
Set 1: [('image6', array([1, 3, 1])), ('image3', array([5, 6, 4])), ('image2', array([5, 3, 2]))]
Set 2: [('image0', array([0, 2, 1])), ('image1', array([2, 3, 6])), ('image4', array([ 5, 16,  1]))]


In [ ]:
# Partition By Median Tests 2
if __name__ == '__main__':
  data = [("img_1",np.array([0,0])), ("img_2",np.array([0,1])), ("img_3",np.array([2,2])), ("img_4",np.array([3,3]))]

  vantage_point_tree = VP_Tree("Index_Test", 5)

  node, s_1, s_2 = vantage_point_tree.partition_by_median(data)

  print("Pivot:", node.pivot)
  print("Median:", node.median)
  print("Set 1:", s_1)
  print("Set 2:", s_2)

Pivot: ('img_4', array([3, 3]))
Median: 3.605551275463989
Set 1: [('img_3', array([2, 2])), ('img_2', array([0, 1]))]
Set 2: [('img_2', array([0, 1])), ('img_1', array([0, 0]))]


### Tests on creation

In [ ]:
if __name__ == '__main__':
  # Create Tree Test 1
  vantage_point_tree = VP_Tree("Index_Test_1",4)
  vantage_point_tree.create_vptree(FEATURES_NAMES_TEST_1,FEATURES_PATH_TEST_1)
  result = VP_Tree.print_tree(vantage_point_tree.root, level=1)

  print("Result:\n" + result + "\n")

  index_name = vantage_point_tree.index_name

  path = VP_Tree.save_vptree(os.path.join(TEST_PATH, index_name),vantage_point_tree)

  print("Saved on:", path)

  tree_loaded = VP_Tree.load_vptree(path)

  print("\nTree Loaded\n")
  result = VP_Tree.print_tree(tree_loaded.root, level=1)
  print(result)

Number of data: 21
The max height of the tree is: 3
Level n.1
Level n.2
Level n.2
Building of the tree completed in: 7.096553802490234 s
Result:
id: 0 ('img16', array([ 1, 30, 60]))
	id: 00 ('img1', array([2, 3, 6]))
		id: 000 ('img11', array([ 0, 11, 21]))
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_1/leaves_3/000_subset_1.npy
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_1/leaves_3/000_subset_2.npy
		id: 001 ('img9', array([15, 32, 22]))
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_1/leaves_3/001_subset_1.npy
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_1/leaves_3/001_subset_2.npy
	id: 01 ('img4', array([ 5, 16,  1]))
		id: 010 ('img2', array([5, 3, 2]))
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_1/leaves_3/010_subset_1.npy
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_1/leaves_3/010_subset_2.npy
		id: 011 ('img

In [ ]:
# Create Tree Test 2
if __name__ == '__main__':
  vantage_point_tree = VP_Tree("Index_Test_2",4)
  vantage_point_tree.create_vptree(FEATURES_NAMES_TEST_2, FEATURES_PATH_TEST_2)
  result = VP_Tree.print_tree(vantage_point_tree.root, level=1)
  print("Result:\n" + result + "\n")

  path = VP_Tree.save_vptree(os.path.join(TEST_PATH, index_name),vantage_point_tree)

  print("Saved on:", path)

  tree_loaded = VP_Tree.load_vptree(path)

  print("\nTree Loaded\n")
  result = VP_Tree.print_tree(tree_loaded.root, level=1)
  print(result)

Number of data: 30
The max height of the tree is: 3
Level n.1
Level n.2
Level n.2
Building of the tree completed in: 5.280122995376587 s
Result:
id: 0 ('img_22', array([92., 86., 13.]))
	id: 00 ('img_8', array([98., 27., 22.]))
		id: 000 ('img_0', array([63., 27., 38.]))
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_2/leaves_3/000_subset_1.npy
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_2/leaves_3/000_subset_2.npy
		id: 001 ('img_11', array([62., 56., 63.]))
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_2/leaves_3/001_subset_1.npy
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_2/leaves_3/001_subset_2.npy
	id: 01 ('img_24', array([26.,  3., 10.]))
		id: 010 ('img_5', array([46., 19., 41.]))
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_2/leaves_3/010_subset_1.npy
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_2/leaves_3/0

In [ ]:
# Create Tree Test 3
if __name__ == '__main__':
  vantage_point_tree = VP_Tree("Index_Test_3",6)
  vantage_point_tree.create_vptree(FEATURES_NAMES_TEST_3, FEATURES_PATH_TEST_3)
  result = VP_Tree.print_tree(vantage_point_tree.root, level=1)
  print("Result:\n" + result + "\n")

  path = VP_Tree.save_vptree(os.path.join(TEST_PATH, index_name),vantage_point_tree)

  print("Saved on:", path)

  tree_loaded = VP_Tree.load_vptree(path)

  print("\nTree Loaded\n")
  result = VP_Tree.print_tree(tree_loaded.root, level=1)
  print(result)

Number of data: 100
The max height of the tree is: 5
Level n.1
Level n.2
Level n.3
Level n.4
Level n.4
Level n.3
Level n.4
Level n.4
Level n.2
Level n.3
Level n.4
Level n.4
Level n.3
Level n.4
Level n.4
Building of the tree completed in: 17.37715744972229 s
Result:
id: 0 ('img_56', array([91, 67, 45, 21]))
	id: 00 ('img_92', array([85, 54, 43, 78]))
		id: 000 ('img_57', array([69, 12, 60, 52]))
			id: 0000 ('img_53', array([53, 49, 72, 46]))
				id: 00000 ('img_33', array([45, 51, 30, 44]))
					/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_3/leaves_5/00000_subset_1.npy
					/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_3/leaves_5/00000_subset_2.npy
				id: 00001 ('img_1', array([73, 40, 13, 47]))
					/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_3/leaves_5/00001_subset_1.npy
					/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_3/leaves_5/00001_subset_2.npy
			id: 0001 ('img_88', array([4

In [ ]:
# Create Tree Test 4
if __name__ == '__main__':
  vantage_point_tree = VP_Tree("Index_Test_4",4, disk_mode=True,leaves_path=TEST_PATH)
  vantage_point_tree.create_vptree(FEATURES_NAMES_TEST_1, FEATURES_PATH_TEST_1)
  result = VP_Tree.print_tree(vantage_point_tree.root, level=1, disk_mode=True)

  print("Result:\n" + result + "\n")

  path = VP_Tree.save_vptree(os.path.join(TEST_PATH, index_name),vantage_point_tree)

  print("Saved on:", path)

  tree_loaded = VP_Tree.load_vptree(path)

  print("\nTree Loaded\n")
  result = VP_Tree.print_tree(tree_loaded.root, level=1)
  print(result)

Number of data: 21
The max height of the tree is: 3
Level n.1
Level n.2
Level n.2
Building of the tree completed in: 4.337145090103149 s
Result:
id: 0 ('img3', array([5, 6, 4]))
	id: 00 ('img1', array([2, 3, 6]))
		id: 000 ('img0', array([0, 2, 1]))
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_4/leaves_3/000_subset_1.npy
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_4/leaves_3/000_subset_2.npy
		id: 001 ('img4', array([ 5, 16,  1]))
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_4/leaves_3/001_subset_1.npy
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_4/leaves_3/001_subset_2.npy
	id: 01 ('img19', array([ 2, 40,  2]))
		id: 010 ('img17', array([25,  6, 41]))
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_4/leaves_3/010_subset_1.npy
			/content/gdrive/My Drive/[MIRCV]FoodWebSearch/antonio-tests/Index_Test_4/leaves_3/010_subset_2.npy
		id: 011 ('img10'

### Tests on Searching

In [22]:
# k-NN search Test 1
if __name__ == '__main__':
  vantage_point_tree = VP_Tree("Index_Test",4)
  vantage_point_tree.create_vptree(FEATURES_NAMES_TEST_1, FEATURES_PATH_TEST_1)

  query = [0,1,1]
  k = 3

  print("Query:", query, "\n")

  nn, d_nn = vantage_point_tree.knn_search(k, query)

  print("\nk-NN search on the tree")
  print("Points:", nn)
  print("Distances:", d_nn)

  print("\n\nSequential Scan on the array")
  data = np.load(FEATURES_PATH_TEST_1)
  distances = [d.euclidean(point, query) for point in data]
  zipped = sorted(zip(data, distances), key=lambda x:x[1])
  for element in zipped[0:k]:
    print(element)


Number of data: 21
The max height of the tree is: 3
Level n.1
Level n.2
Level n.2
Building of the tree completed in: 0.052635908126831055 s
Query: [0, 1, 1] 

[inf, inf, inf]
[inf, inf, 5.744562646538029]
[inf, 231.97629189208106, 5.744562646538029]
[39.06404996924922, 32.87856444554719, 5.744562646538029]
[39.06404996924922, 32.87856444554719, 5.744562646538029]
[32.87856444554719, 7.681145747868608, 5.744562646538029]
[9.433981132056603, 7.681145747868608, 5.744562646538029]
Query [0, 1, 1] answered in 0.01772022247314453  s

k-NN search on the tree
Points: [array(['img0', array([0, 2, 1])], dtype=object), array(['img6', array([1, 3, 1])], dtype=object), ('img5', array([2, 6, 2]))]
Distances: [1.0, 2.23606797749979, 5.477225575051661]


Sequential Scan on the array
(array([0, 2, 1]), 1.0)
(array([1, 3, 1]), 2.23606797749979)
(array([5, 3, 2]), 5.477225575051661)


In [23]:
# k-NN search Test 2
if __name__ == '__main__':
  vantage_point_tree = VP_Tree("Index_Test_3",4)
  vantage_point_tree.create_vptree(FEATURES_NAMES_TEST_2, FEATURES_PATH_TEST_2)

  query = [0,1,1]
  k = 3

  nn, d_nn = vantage_point_tree.knn_search(k, query)

  print("Query:", query)

  print("k-NN search on the tree")
  print("Points:", nn)
  print("Distances:", d_nn)

  print("\n\nSequential Scan on the array")
  data = np.load(FEATURES_PATH_TEST_2)
  distances = [d.euclidean(point, query) for point in data]
  zipped = sorted(zip(data, distances), key=lambda x:x[1])
  for element in zipped[0:k]:
    print(element)

Number of data: 30
The max height of the tree is: 3
Level n.1
Level n.2
Level n.2
Building of the tree completed in: 2.0386343002319336 s
[inf, inf, inf]
[inf, inf, 110.22703842524301]
[inf, 110.22703842524301, 27.586228448267445]
[103.54226190305097, 100.80674580602232, 27.586228448267445]
[34.539832078341085, 27.586228448267445, 22.825424421026653]
[34.539832078341085, 27.586228448267445, 22.825424421026653]
Query [0, 1, 1] answered in 0.011181354522705078  s
Query: [0, 1, 1]
k-NN search on the tree
Points: [array(['img_10', array([ 4.,  9., 22.])], dtype=object), ('img_24', array([26.,  3., 10.])), array(['img_6', array([32., 13.,  6.])], dtype=object)]
Distances: [22.825424421026653, 27.586228448267445, 34.539832078341085]


Sequential Scan on the array
(array([ 4.,  9., 22.]), 22.825424421026653)
(array([26.,  3., 10.]), 27.586228448267445)
(array([32., 13.,  6.]), 34.539832078341085)


In [24]:
# k-NN search Test 3
if __name__ == '__main__':

  vantage_point_tree = VP_Tree("Index_Test_3",4)
  vantage_point_tree.create_vptree(FEATURES_NAMES_TEST_3, FEATURES_PATH_TEST_3)

  query = [0,1,1,2]
  k = 6

  nn, d_nn = vantage_point_tree.knn_search(k, query)

  print("Query:", query)

  print("k-NN search on the tree")
  print("Points:", nn)
  print("Distances:", d_nn)

  print("\n\nSequential Scan on the array")
  data = np.load(FEATURES_PATH_TEST_3)
  distances = [d.euclidean(point, query) for point in data]
  zipped = sorted(zip(data, distances), key=lambda x:x[1])
  for element in zipped[0:k]:
    print(element)

Number of data: 100
The max height of the tree is: 5
Level n.1
Level n.2
Level n.3
Level n.3
Level n.2
Level n.3
Level n.3
Building of the tree completed in: 3.8324925899505615 s
[inf, inf, inf, inf, inf, inf]
[inf, inf, inf, inf, inf, 125.40334923757021]
[inf, inf, inf, inf, 125.40334923757021, 81.93900170248598]
[inf, inf, inf, 125.40334923757021, 121.97130810153674, 81.93900170248598]
[106.44247272588137, 101.91172650877817, 100.24470060806208, 97.2265395866787, 81.93900170248598, 32.72613634390714]
[101.91172650877817, 100.24470060806208, 97.2265395866787, 81.93900170248598, 76.75936424958195, 32.72613634390714]
[100.24470060806208, 97.2265395866787, 85.03528679318957, 81.93900170248598, 76.75936424958195, 32.72613634390714]
[76.86351540230254, 76.75936424958195, 76.58328799418317, 56.7626637852735, 48.14561246884289, 32.72613634390714]
[49.8196748283246, 48.68264577855234, 48.14561246884289, 45.110974274559844, 40.95119045888654, 32.72613634390714]
[49.8196748283246, 48.6826457785

In [25]:
#k-NN search Test 4
if __name__ == '__main__':
  vantage_point_tree = VP_Tree("Index_Test_3",4, disk_mode=True)
  vantage_point_tree.create_vptree(FEATURES_NAMES_TEST_3, FEATURES_PATH_TEST_3)

  query = [0,1,1,2]
  k = 6

  nn, d_nn = vantage_point_tree.knn_search(k, query)

  print("Query:", query)

  print("k-NN search on the tree")
  print("Points:", nn)
  print("Distances:", d_nn)

  print("\n\nSequential Scan on the array")
  data = np.load(FEATURES_PATH_TEST_3)
  distances = [d.euclidean(point, query) for point in data]
  zipped = sorted(zip(data, distances), key=lambda x:x[1])
  for element in zipped[0:k]:
    print(element)

Number of data: 100
The max height of the tree is: 5
Level n.1
Level n.2
Level n.3
Level n.3
Level n.2
Level n.3
Level n.3
Building of the tree completed in: 0.13038420677185059 s
[inf, inf, inf, inf, inf, inf]
[inf, inf, inf, inf, inf, 79.27168473042566]
[inf, inf, inf, inf, 148.49242404917499, 79.27168473042566]
[inf, inf, inf, 148.49242404917499, 103.9038016628843, 79.27168473042566]
[120.04998958767135, 118.0762465528101, 103.9038016628843, 94.64671151181112, 79.27168473042566, 76.86351540230254]
[91.00549433962765, 83.40863264674707, 83.40863264674707, 81.93900170248598, 79.27168473042566, 76.86351540230254]
[91.00549433962765, 83.40863264674707, 83.40863264674707, 81.93900170248598, 79.27168473042566, 76.86351540230254]
[91.00549433962765, 83.40863264674707, 83.40863264674707, 81.93900170248598, 79.27168473042566, 76.86351540230254]
[91.00549433962765, 83.40863264674707, 83.40863264674707, 81.93900170248598, 79.27168473042566, 76.86351540230254]
[83.40863264674707, 83.40863264674

#Deployment

In [26]:
if __name__ == '__main__':
  vantage_point_tree = VP_Tree("index",height=10, disk_mode=True, leaves_path=INDEX_DIR)
  vantage_point_tree.create_vptree(ID_DEPLOYMENT, FEATURES_DEPLOYMENT)

  index_name = vantage_point_tree.index_name
  dest_folder = os.path.join(INDEX_DIR, index_name)
  print("Destination Folder: ", dest_folder)

  VP_Tree.save_vptree(dest_folder,vantage_point_tree)

Number of data: 125997
The max height of the tree is: 15
Level n.1
Level n.2
Level n.3
Level n.4
Level n.5
Level n.6
Level n.7
Level n.8
Level n.9
Level n.9
Level n.8
Level n.9
Level n.9
Level n.7
Level n.8
Level n.9
Level n.9
Level n.8
Level n.9
Level n.9
Level n.6
Level n.7
Level n.8
Level n.9
Level n.9
Level n.8
Level n.9
Level n.9
Level n.7
Level n.8
Level n.9
Level n.9
Level n.8
Level n.9
Level n.9
Level n.5
Level n.6
Level n.7
Level n.8
Level n.9
Level n.9
Level n.8
Level n.9
Level n.9
Level n.7
Level n.8
Level n.9
Level n.9
Level n.8
Level n.9
Level n.9
Level n.6
Level n.7
Level n.8
Level n.9
Level n.9
Level n.8
Level n.9
Level n.9
Level n.7
Level n.8
Level n.9
Level n.9
Level n.8
Level n.9
Level n.9
Level n.4
Level n.5
Level n.6
Level n.7
Level n.8
Level n.9
Level n.9
Level n.8
Level n.9
Level n.9
Level n.7
Level n.8
Level n.9
Level n.9
Level n.8
Level n.9
Level n.9
Level n.6
Level n.7
Level n.8
Level n.9
Level n.9
Level n.8
Level n.9
Level n.9
Level n.7
Level n.8
Level n.9
Lev

In [ ]:
if __name__ == '__main__':
  dir = os.path.join(INDEX_DIR,vantage_point_tree.index_name)
  dir = os.path.join(dir, "leaves")
  leaf = np.load(os.path.join(dir, "1111111111_subset_1.npy"), allow_pickle=True)
  print("Objects per leaf:", len(leaf))

123


In [31]:
if __name__ == '__main__':
  dest_folder = os.path.join(INDEX_DIR, "index/index.json")
  vantage_point_tree = VP_Tree.load_vptree(dest_folder)
  query = [random.randint(-50,50) for _ in range(1280)]
  start = time.time()
  result = vantage_point_tree.knn_search(k=10, query=query)
  end = time.time()
  k_images = [element[0] for element in result[0]]
  print(k_images)
  print("Ricerca effettuata in:", end - start, "s")

Tree loaded correctly
Query [40, 39, 7, -12, 24, -29, 18, -3, 34, 3, -2, -17, 20, -14, -9, -21, 9, -3, 42, -18, -16, -50, -48, -15, -9, -35, 36, 15, -45, -30, -6, -12, -27, -7, 18, -14, 42, -9, 12, 28, -15, -50, -26, 19, 8, -26, -11, -44, -17, -28, -47, -32, 8, 11, -26, 41, 18, 0, -31, -2, 40, -41, 29, -22, -42, -16, -33, 34, -11, 34, -47, 41, 25, -46, -41, 20, -36, -24, -46, 17, -5, 37, 44, -6, 30, 12, 16, -14, 19, 22, 28, -36, -6, 24, 42, -21, -21, -40, -37, 20, -44, 28, -45, 36, -12, 45, 47, -4, -5, 5, -15, 34, 44, 29, 9, 11, 20, -46, 47, -14, 25, -31, -35, -18, -32, -5, 14, -30, -25, -50, -19, -2, -16, -23, -49, -18, -3, 43, 29, -42, 28, 25, -6, -11, 25, -26, -23, -25, 23, -26, -37, 37, -1, 1, 41, -40, -7, 0, 34, 1, 37, -41, 27, -1, 8, 28, 3, -44, 5, -46, -44, 42, -37, -10, 38, 34, 16, 22, 26, -6, 36, -24, 30, 39, -42, 21, 37, -16, 0, 38, 19, -3, 20, 42, 38, -8, 49, 23, 38, -35, -50, 33, -10, -50, -22, -27, 11, 9, 30, 24, -20, -41, 14, -39, 40, 31, 19, -32, -19, -6, 38, -44, -5, 32